<a href="https://colab.research.google.com/github/JC230903/ml_projects/blob/main/EarthquakeDeletction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pytorch implementation in python

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
df=pd.read_csv("/content/sample_data/earthquakes.csv")

In [ ]:
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,Alert
0,1976-03-25 00:41:20.500000+00:00,35.590000,-90.480000,15.000,4.62,mw,NaN,NaN,NaN,NaN,...,"8 km NW of Marked Tree, Arkansas",earthquake,NaN,NaN,NaN,0.0,reviewed,nm,nm,green
1,1979-10-16 06:58:43.450000+00:00,32.998667,-115.557500,14.190,5.80,ml,7.0,79.0,0.05768,0.17,...,"3km NW of Brawley, CA",earthquake,1.03,1.78,NaN,0.0,reviewed,ci,ci,yellow
2,1980-07-27 18:52:21.600000+00:00,38.190000,-83.950000,10.000,5.00,md,NaN,NaN,NaN,NaN,...,"2 km SW of Sharpsburg, Kentucky",earthquake,NaN,NaN,NaN,NaN,reviewed,se,se,green
3,1981-04-19 09:02:50.520000+00:00,35.816000,-117.816333,4.766,4.70,ml,13.0,135.0,NaN,0.66,...,"16km SSE of Little Lake, CA",earthquake,2.51,31.61,0.424,9.0,reviewed,ci,ci,green
4,1981-04-26 12:09:28.290000+00:00,33.095500,-115.624500,18.904,5.75,ml,81.0,34.0,NaN,0.34,...,"The 1981 Westmorland, California Earthquake",earthquake,0.56,0.67,0.161,6.0,reviewed,ci,ci,green


In [ ]:
df["Alert"].sum()

'...'

In [ ]:
df['Alert'] = df['Alert'].map({'green': 1, 'yellow': 0})
print(df)

                                  time   latitude   longitude   depth   mag  \
0     1976-03-25 00:41:20.500000+00:00  35.590000  -90.480000  15.000  4.62   
1     1979-10-16 06:58:43.450000+00:00  32.998667 -115.557500  14.190  5.80   
2     1980-07-27 18:52:21.600000+00:00  38.190000  -83.950000  10.000  5.00   
3     1981-04-19 09:02:50.520000+00:00  35.816000 -117.816333   4.766  4.70   
4     1981-04-26 12:09:28.290000+00:00  33.095500 -115.624500  18.904  5.75   
...                                ...        ...         ...     ...   ...   
7712  2024-12-19 02:07:02.552000+00:00 -56.357400  147.017000  10.000  5.70   
7713  2024-12-20 14:33:08.325000+00:00 -11.751300  166.170800  25.375  5.40   
7714  2024-12-21 15:30:53.243000+00:00 -17.709800  168.029200  46.000  6.10   
7715  2024-12-22 03:56:04.489000+00:00  55.982500 -153.997300   9.200  5.60   
7716  2024-12-23 06:00:59.074000+00:00  19.916200  -76.515300  22.241  5.90   

     magType    nst    gap      dmin   rms  ...  \


In [ ]:
df.describe()

,latitude,longitude,depth,mag,nst,gap,dmin,rms,horizontalError,depthError,magError,magNst,Alert
count,7717.000000,7717.000000,7717.000000,7717.000000,2404.000000,7393.000000,7015.000000,7691.000000,6499.000000,7508.000000,5486.000000,5650.000000,7632.000000
mean,1.045090,14.813158,58.852818,5.656460,147.059900,49.049605,4.099935,0.820820,6.956036,2.578836,0.067912,46.789558,0.967767
std,32.336895,128.439465,119.054871,0.514043,116.907505,39.095080,5.256580,0.267801,2.825249,3.171284,0.057431,70.149373,0.176629
min,-69.773900,-179.977600,-1.770000,4.500000,0.000000,7.000000,0.000000,0.040000,0.080000,0.000000,0.000000,0.000000,0.000000
25%,-21.720500,-110.836000,10.000000,5.400000,75.000000,25.000000,1.035000,0.660000,5.555000,1.700000,0.047000,18.000000,1.000000
50%,-3.433600,40.021100,13.540000,5.600000,119.000000,38.000000,2.399000,0.820000,7.100000,1.800000,0.059000,29.000000,1.000000
75%,28.199800,141.086700,42.000000,5.900000,182.000000,59.000000,4.957500,0.990000,8.600000,2.900000,0.073000,47.000000,1.000000
max,85.729000,179.998100,670.810000,8.600000,770.000000,321.000000,39.934000,2.100000,88.540000,31.950000,1.642000,954.000000,1.000000


In [ ]:
df.shape

(7717, 23)

In [ ]:
df.columns

Index(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst',
       'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type',
       'horizontalError', 'depthError', 'magError', 'magNst', 'status',
       'locationSource', 'magSource', 'Alert'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7717 entries, 0 to 7716
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   time             7717 non-null   object 
 1   latitude         7717 non-null   float64
 2   longitude        7717 non-null   float64
 3   depth            7717 non-null   float64
 4   mag              7717 non-null   float64
 5   magType          7717 non-null   object 
 6   nst              2404 non-null   float64
 7   gap              7393 non-null   float64
 8   dmin             7015 non-null   float64
 9   rms              7691 non-null   float64
 10  net              7717 non-null   object 
 11  id               7717 non-null   object 
 12  updated          7717 non-null   object 
 13  place            7717 non-null   object 
 14  type             7717 non-null   object 
 15  horizontalError  6499 non-null   float64
 16  depthError       7508 non-null   float64
 17  magError      

In [ ]:
columns_names=['horizontalError','depthError','magError','magNst','nst','gap','dmin','rms']


df.drop(columns=columns_names,inplace=True)


In [ ]:
df= df.dropna(subset=['Alert'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7632 entries, 0 to 7716
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   time            7632 non-null   object 
 1   latitude        7632 non-null   float64
 2   longitude       7632 non-null   float64
 3   depth           7632 non-null   float64
 4   mag             7632 non-null   float64
 5   magType         7632 non-null   object 
 6   net             7632 non-null   object 
 7   id              7632 non-null   object 
 8   updated         7632 non-null   object 
 9   place           7632 non-null   object 
 10  type            7632 non-null   object 
 11  status          7632 non-null   object 
 12  locationSource  7632 non-null   object 
 13  magSource       7632 non-null   object 
 14  Alert           7632 non-null   float64
dtypes: float64(5), object(10)
memory usage: 954.0+ KB


In [ ]:
# Drop all columns of type object
df = df.select_dtypes(exclude=['object'])

print(df)


       latitude   longitude   depth   mag  Alert
0     35.590000  -90.480000  15.000  4.62    1.0
1     32.998667 -115.557500  14.190  5.80    0.0
2     38.190000  -83.950000  10.000  5.00    1.0
3     35.816000 -117.816333   4.766  4.70    1.0
4     33.095500 -115.624500  18.904  5.75    1.0
...         ...         ...     ...   ...    ...
7712 -56.357400  147.017000  10.000  5.70    1.0
7713 -11.751300  166.170800  25.375  5.40    1.0
7714 -17.709800  168.029200  46.000  6.10    1.0
7715  55.982500 -153.997300   9.200  5.60    1.0
7716  19.916200  -76.515300  22.241  5.90    1.0

[7632 rows x 5 columns]


In [ ]:
X_ = df[['latitude', 'longitude', 'depth', 'mag']]

In [ ]:
print(X_)

       latitude   longitude   depth   mag
0     35.590000  -90.480000  15.000  4.62
1     32.998667 -115.557500  14.190  5.80
2     38.190000  -83.950000  10.000  5.00
3     35.816000 -117.816333   4.766  4.70
4     33.095500 -115.624500  18.904  5.75
...         ...         ...     ...   ...
7712 -56.357400  147.017000  10.000  5.70
7713 -11.751300  166.170800  25.375  5.40
7714 -17.709800  168.029200  46.000  6.10
7715  55.982500 -153.997300   9.200  5.60
7716  19.916200  -76.515300  22.241  5.90

[7632 rows x 4 columns]


In [ ]:
y_= df['Alert']

In [ ]:
print(y_)

0       1.0
1       0.0
2       1.0
3       1.0
4       1.0
       ... 
7712    1.0
7713    1.0
7714    1.0
7715    1.0
7716    1.0
Name: Alert, Length: 7632, dtype: float64


In [ ]:
X_ = df[['latitude', 'longitude', 'depth', 'mag']].to_numpy()
y_ = df['Alert'].to_numpy()
print(X_.shape)  # Should print (7632, 4)
print(y_.shape)

(7632, 4)
(7632,)


In [ ]:
X = torch.from_numpy(X_).type(torch.float)
y = torch.from_numpy(y_).type(torch.float)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2, # 20% test, 80% train
                                                    random_state=42) # make the random split reproducible

len(X_train), len(X_test), len(y_train), len(y_test)

(6105, 1527, 6105, 1527)

In [ ]:
y.shape

torch.Size([7632])

In [ ]:
X.shape

torch.Size([7632, 4])

In [ ]:
if torch.backends.mps.is_available():
    device = "mps"  # Use Metal Performance Shaders (for Apple GPUs)
elif torch.cuda.is_available():
    device = "cuda"  # Use CUDA (for NVIDIA GPUs)
else:
    device = "cpu"  # Default to CPU

print(f"Using device: {device}")

Using device: cpu


In [ ]:
class classModel(nn.Module):
  def __init__(self):
      super().__init__()

      self.layer1=nn.Linear(in_features=4,out_features=16)
      self.layer2=nn.Linear(in_features=16,out_features=64)
      self.layer3=nn.Linear(in_features=64,out_features=1)
      self.relu = nn.ReLU()


  def forward(self,x):
       x = self.layer1(x)
       x = self.relu(x)
       x = self.layer2(x)
       x = self.layer3(x)
       return x









In [ ]:
model1=classModel().to(device)
model1

classModel(
  (layer1): Linear(in_features=4, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

In [ ]:
untrain_pred=model1(X_test.to(device))
print(f"length of predictions {len(untrain_pred)} shape of data is {untrain_pred.shape}")
print(f"Length of test samples: {len(y_test)}, Shape: {y_test.shape}")
print(f"\nFirst 10 predictions:\n{untrain_pred[:10]}")
print(f"\nFirst 10 test labels:\n{y_test[:10]}")


length of predictions 1527 shape of data is torch.Size([1527, 1])
Length of test samples: 1527, Shape: torch.Size([1527])

First 10 predictions:
tensor([[-0.5794],
        [10.9543],
        [ 1.4618],
        [17.1305],
        [ 1.0800],
        [10.3403],
        [ 8.3669],
        [26.0723],
        [11.6939],
        [ 1.5368]], grad_fn=<SliceBackward0>)

First 10 test labels:
tensor([1., 1., 1., 1., 1., 0., 1., 1., 1., 1.])


In [ ]:
loss_fn = nn.BCEWithLogitsLoss()  # Binary Cross Entropy Loss with logits
optimizer=torch.optim.Adam(model1.parameters(),lr=0.001)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

training loop and testing loop

In [ ]:
num_epochs = 500

for epoch in range(num_epochs):
    model1.train()  # Set the model to training mode

    # Ensure X_train is on the correct device
    X_train, y_train = X_train.to(device), y_train.to(device)

    # Forward pass
    y_logits = model1(X_train).squeeze()  # Get the raw predictions (logits)
    y_pred = torch.round(torch.sigmoid(y_logits))  # Convert logits to probabilities, then round to 0 or 1

    # Compute training loss
    loss = loss_fn(y_logits, y_train)  # Use logits directly with appropriate loss (e.g., BCEWithLogitsLoss)

    # Compute training accuracy
    acc = accuracy_fn(y_true=y_train.cpu(), y_pred=y_pred.cpu())  # Detach to compute in CPU

    # Backward pass and optimization
    optimizer.zero_grad()  # Clear gradients
    loss.backward()  # Compute gradients
    optimizer.step()  # Update model parameters

    # Logging training progress
    if (epoch + 1) % 100 == 0:  # Log every 10 epochs
        print(f"Epoch {epoch + 1}/{num_epochs} | Loss: {loss.item():.4f} | Accuracy: {acc:.2f}%")

# Evaluation phase
model1.eval()  # Set the model to evaluation mode

with torch.inference_mode():
    # Move test data to the same device
    X_test, y_test = X_test.to(device), y_test.to(device)

    # Forward pass for test data
    test_logits = model1(X_test).squeeze()  # Get logits for test data
    test_loss = loss_fn(test_logits, y_test)  # Compute test loss

    # Convert logits to probabilities and binary predictions
    test_pred = torch.round(torch.sigmoid(test_logits))  # Sigmoid + round for binary classification
    test_acc = accuracy_fn(y_true=y_test.cpu(), y_pred=test_pred.cpu())  # Detach to compute in CPU

# Print test results
print(f"Test Loss: {test_loss.item():.4f} | Test Accuracy: {test_acc:.2f}%")




Epoch 100/500 | Loss: 0.1188 | Accuracy: 96.82%
Epoch 200/500 | Loss: 0.1161 | Accuracy: 96.82%
Epoch 300/500 | Loss: 0.1136 | Accuracy: 96.82%
Epoch 400/500 | Loss: 0.1123 | Accuracy: 96.82%
Epoch 500/500 | Loss: 0.1113 | Accuracy: 96.82%
Test Loss: 0.1168 | Test Accuracy: 96.59%
